In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.utils.data
import torch.nn as nn
import torch.optim as optim
from timeit import default_timer as timer
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

Xtr_path = 'Xtr.csv'
Ytr_path = 'ytr.csv'
Xts_path = 'Xts.csv'

Xtr = np.loadtxt(Xtr_path, delimiter=",")
Ytr = np.loadtxt(Ytr_path, delimiter=",")
Xts = np.loadtxt(Xts_path, delimiter=",")

In [2]:
Xtr,Xts,Ytr,Yts = train_test_split(Xtr,Ytr,test_size=0.2,random_state=0,stratify=Ytr)

In [3]:
Xtr.shape


(8000, 16)

In [4]:
Ytr.shape


(8000,)

In [5]:
Xts.shape

(2000, 16)

In [6]:
Yts.shape

(2000,)

In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

batch_size = 128

# TODO Convert the numpy arrays to tensors
Xtr_torch = torch.Tensor(Xtr)
ytr_torch = torch.Tensor(Ytr)
Xts_torch = torch.Tensor(Xts)
yts_torch = torch.Tensor(Yts)

# TODO Create a training/test dataset from the tensors
# train_ds = ...
# test_ds = ...
train_ds = TensorDataset(Xtr_torch, ytr_torch)
# Create a testing Dataset from the tensors
test_ds = TensorDataset(Xts_torch, yts_torch)


# TODO Create a training/test data loader from datasets
# train_loader = ...
# test_loader = ...
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(test_ds, batch_size=batch_size) 

In [8]:
d_in = Xtr.shape[1]
d_out = len(np.unique(Ytr))
nh=100
class DumbNet(nn.Module):
    def __init__(self):
        super(DumbNet, self).__init__()
        
        self.classifier=nn.Sequential(
            nn.Linear(d_in,165),
            nn.BatchNorm1d(165),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(165,190),
            nn.BatchNorm1d(190),
          
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(190,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
          
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
           
            nn.GELU(),
            nn.Linear(64,d_out), 
            nn.LogSoftmax(dim=1)
            
        
        
      
    )
        
      
       
       
        
      
    def forward(self,x):
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
      
        return x

model = DumbNet()     

In [9]:
print(model)

DumbNet(
  (classifier): Sequential(
    (0): Linear(in_features=16, out_features=165, bias=True)
    (1): BatchNorm1d(165, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): GELU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=165, out_features=190, bias=True)
    (5): BatchNorm1d(190, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): GELU()
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=190, out_features=128, bias=True)
    (9): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.01)
    (11): Linear(in_features=128, out_features=64, bias=True)
    (12): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): GELU()
    (14): Linear(in_features=64, out_features=26, bias=True)
    (15): LogSoftmax(dim=1)
  )
)


In [10]:
with torch.no_grad():
    ytr_net = model(torch.Tensor(Xtr)).detach().numpy().argmax(axis=1)

acc = np.mean(ytr_net==Ytr)
print('training accuracy: ',acc)

training accuracy:  0.035


In [11]:
import torch.optim as optim

# TODO
# opt = ...
# criterion = ...

opt = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

In [12]:
a_tr_loss = []
a_tr_accuracy = []
a_ts_loss = []
a_ts_accuracy = []


num_epoch = 400


for epoch in range(num_epoch):
    model.train() # put model in training mode
    correct = 0 # initialize error counter
    total = 0 # initialize total counter
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        y_batch = y_batch.type(torch.long)
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch)
        a_tr_loss.append(loss.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        # Do hard classification: index of largest score
        _, predicted = torch.max(out.data, 1)
        # Compute number of decision errors
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
    a_tr_accuracy.append( 100*correct/total )
    
    model.eval() # put model in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = labels.type(torch.long)
            outputs = model(images)
            loss = criterion(outputs,labels)
            a_ts_loss.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    a_ts_accuracy.append( 100*correct/total )
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Acc: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
          +'Test Acc: {0:.2f}'.format(a_ts_accuracy[epoch]))
   

Epoch:  1   Train Loss: 3.320   Train Acc: 20.40    Test Loss: 2.164   Test Acc: 42.25
Epoch:  2   Train Loss: 3.318   Train Acc: 42.14    Test Loss: 2.369   Test Acc: 58.55
Epoch:  3   Train Loss: 3.277   Train Acc: 50.60    Test Loss: 2.214   Test Acc: 64.60
Epoch:  4   Train Loss: 3.246   Train Acc: 55.01    Test Loss: 2.148   Test Acc: 66.50
Epoch:  5   Train Loss: 3.260   Train Acc: 58.08    Test Loss: 2.184   Test Acc: 68.90
Epoch:  6   Train Loss: 3.193   Train Acc: 60.12    Test Loss: 2.187   Test Acc: 69.75
Epoch:  7   Train Loss: 3.182   Train Acc: 61.81    Test Loss: 2.086   Test Acc: 72.25
Epoch:  8   Train Loss: 3.189   Train Acc: 63.95    Test Loss: 2.164   Test Acc: 72.10
Epoch:  9   Train Loss: 3.206   Train Acc: 64.90    Test Loss: 2.219   Test Acc: 72.70
Epoch: 10   Train Loss: 3.148   Train Acc: 66.66    Test Loss: 2.056   Test Acc: 75.50
Epoch: 11   Train Loss: 3.163   Train Acc: 67.19    Test Loss: 2.197   Test Acc: 75.55
Epoch: 12   Train Loss: 3.172   Train Acc: 

In [13]:
with torch.no_grad():
    ytr_net = model(torch.Tensor(Xtr)).detach().numpy().argmax(axis=1)

acc = np.mean(ytr_net==Ytr)
print('training accuracy: ',acc)

training accuracy:  0.970875


In [14]:
# save the model using the extension .pth
save_path = 'model.pth'

# To save a PyTorch model, we first pass an input through the model, 
# and then save the "trace". 
# For this purpose, we can use any input. 
# We will create a random input with the proper dimension.
x = torch.randn(d_in) # random input
x = x[None,:] # add singleton batch index
with torch.no_grad():
    traced_cell = torch.jit.trace(model, (x))

# Now we save the trace
torch.jit.save(traced_cell, save_path)


In [15]:
!python {"validation.py " + save_path}

training accuracy =  0.9566
test label predictions saved in yts_hat.csv


In [16]:

d_in = Xtr.shape[1]
d_out = len(np.unique(Ytr))
nh=100
class DumbNet(nn.Module):
    def __init__(self):
        super(DumbNet, self).__init__()
        
        self.classifier=nn.Sequential(
            nn.Linear(d_in,565),
            nn.BatchNorm1d(565),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(565,390),
            nn.BatchNorm1d(390),
          
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(390,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
          
            nn.Linear(128,64),
            nn.BatchNorm1d(64),
           
            nn.GELU(),
            nn.Linear(64,d_out), 
            nn.LogSoftmax(dim=1)
            
        
        
      
    )
        
      
       
       
        
      
    def forward(self,x):
        x=x.view(x.size(0),-1)
        x=self.classifier(x)
      
        return x

model = DumbNet()  
import torch.optim as optim

# TODO
# opt = ...
# criterion = ...

opt = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()
a_tr_loss = []
a_tr_accuracy = []
a_ts_loss = []
a_ts_accuracy = []


num_epoch = 400


for epoch in range(num_epoch):
    model.train() # put model in training mode
    correct = 0 # initialize error counter
    total = 0 # initialize total counter
    # iterate over training set
    for train_iter, data in enumerate(train_loader):
        x_batch,y_batch = data
        y_batch = y_batch.type(torch.long)
        out = model(x_batch)
        # Compute Loss
        loss = criterion(out,y_batch)
        a_tr_loss.append(loss.item())
        # Compute gradients using back propagation
        opt.zero_grad()
        loss.backward()
        # Take an optimization 'step'
        opt.step()
        # Do hard classification: index of largest score
        _, predicted = torch.max(out.data, 1)
        # Compute number of decision errors
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()
        
    a_tr_accuracy.append( 100*correct/total )
    
    model.eval() # put model in evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            labels = labels.type(torch.long)
            outputs = model(images)
            loss = criterion(outputs,labels)
            a_ts_loss.append(loss.item())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    a_ts_accuracy.append( 100*correct/total )
    # Print details every print_mod epoch
    print('Epoch: {0:2d}   Train Loss: {1:.3f}   '.format(epoch+1, a_tr_loss[epoch])
          +'Train Acc: {0:.2f}    Test Loss: {1:.3f}   '.format(a_tr_accuracy[epoch], a_ts_loss[epoch])
          +'Test Acc: {0:.2f}'.format(a_ts_accuracy[epoch]))
   

Epoch:  1   Train Loss: 3.337   Train Acc: 33.91    Test Loss: 1.790   Test Acc: 56.20
Epoch:  2   Train Loss: 3.198   Train Acc: 55.15    Test Loss: 1.990   Test Acc: 64.85
Epoch:  3   Train Loss: 3.173   Train Acc: 60.67    Test Loss: 1.931   Test Acc: 67.65
Epoch:  4   Train Loss: 3.170   Train Acc: 64.75    Test Loss: 1.783   Test Acc: 70.85
Epoch:  5   Train Loss: 3.158   Train Acc: 66.53    Test Loss: 1.765   Test Acc: 72.85
Epoch:  6   Train Loss: 3.079   Train Acc: 68.36    Test Loss: 1.811   Test Acc: 74.40
Epoch:  7   Train Loss: 3.051   Train Acc: 69.24    Test Loss: 1.694   Test Acc: 74.60
Epoch:  8   Train Loss: 3.090   Train Acc: 72.24    Test Loss: 1.827   Test Acc: 77.20
Epoch:  9   Train Loss: 3.013   Train Acc: 72.91    Test Loss: 1.873   Test Acc: 77.30
Epoch: 10   Train Loss: 2.957   Train Acc: 73.90    Test Loss: 1.705   Test Acc: 78.05
Epoch: 11   Train Loss: 2.891   Train Acc: 75.08    Test Loss: 1.823   Test Acc: 78.35
Epoch: 12   Train Loss: 2.845   Train Acc: 

In [17]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# define device CNN
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device("cpu")
    print("cuda is not available")
class ProjectDataset(Dataset):

    def __init__(self, path1):
        super(ProjectDataset, self).__init__()
        self.path1 = path1
        self.dataset1 = []
        self.dataset1.extend(open(os.path.join(self.path1, 'dataset.txt')).readlines())

    def __len__(self):
        return len(self.dataset1)

    def __getitem__(self, index):
        str1 = self.dataset1[index]
        input = []
        for i in str1.split('\t'):
            input.append(float(i))
        number = torch.tensor(input)
        Input = number[:16].reshape(4,4).unsqueeze(0)
        label = number[16:]
        return (Input, label)
def loadData(path_dir, batch_size, shuffle=False):
    # nomalization
    transform = transforms.Compose([
        transforms.ToTensor(),  # (H, W, C) -> (C, H, W) & (0, 255) -> (0, 1)
        # transforms.Normalize(mean=(0.5), std=(0.5))  # (0, 1) -> (-1, 1)
    ])
    dataset = ProjectDataset(path_dir)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

batch_size = 128
path1 = os.getcwd()
train_loader = loadData(path1, batch_size, shuffle=True)

# define CNN
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.leakyrelu = nn.LeakyReLU(0.2, inplace=True)
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=128, kernel_size=3, stride=1, padding=1)
    self.bn1 = nn.BatchNorm2d(128)
    self.dropout1 = nn.Dropout2d(p=0.5)
    self.conv2 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=2, padding=1)
    self.bn2 = nn.BatchNorm2d(128)
    self.dropout2 = nn.Dropout2d(p=0.5)
    self.fc1 = nn.Linear(128*4, 128)
    self.dropout3 = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(128, 26)

  def forward(self, x):
        x = self.conv1(x)
        x = self.leakyrelu(x)
        x = self.bn1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.leakyrelu(x)
        x = self.bn2(x)
        x = self.dropout2(x)
        x = self.fc1(x.view(x.size(0),-1))
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

#主函数
def main():

    # 网络参数
    epoch = 300
    beta1 = 0.8
    lr = 0.001
    decay = lr / epoch

    # 初始化网络、定义loss、optimizer
    net = CNN().to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(net.parameters(), betas=(beta1, 0.999), lr=lr)
    lambda1 = lambda epoch: 1 / (1 + decay * epoch)
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

    # 主循环
    count = 0

    f1 = open(r'loss.txt', 'a')
    f2 = open(r'accuracy.txt', 'a')
    for i in range(epoch):
        total = 0
        correct = 0
        net.train()
        for img, label in train_loader:
            img = img.to(device)
            #print(img.shape)
            #print(label.shape)
            label = label.to(device)
            output = net(img)
            loss = criterion(output, label.squeeze().long())
            # 损失函数的可视化
            # writer.add_scalar('Loss\train',loss,epoch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Compute Accuracy
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label.squeeze()).sum().item()
            s1 = str('%.3f' % (loss.item()))
            f1.write(s1 + '\n')
        accu = 100 * correct / total
        accu1 = accu/100
        f2.write(str(accu1)+ '\n')

        scheduler.step()
        # print(accu)
        print('Epoch: {0:2d}  trainAccuracy: {1:.3f}%' \
            .format(i + 1, accu))
    f1.close()
    f2.close()
    torch.save(net, 'CNN.pkl')

if __name__ == '__main__':
    main()




cuda is not available
Epoch:  1  trainAccuracy: 33.980%
Epoch:  2  trainAccuracy: 49.550%
Epoch:  3  trainAccuracy: 54.880%
Epoch:  4  trainAccuracy: 57.130%
Epoch:  5  trainAccuracy: 58.960%
Epoch:  6  trainAccuracy: 59.570%
Epoch:  7  trainAccuracy: 61.280%
Epoch:  8  trainAccuracy: 61.280%
Epoch:  9  trainAccuracy: 61.850%
Epoch: 10  trainAccuracy: 61.770%
Epoch: 11  trainAccuracy: 63.090%
Epoch: 12  trainAccuracy: 63.450%
Epoch: 13  trainAccuracy: 63.300%
Epoch: 14  trainAccuracy: 63.570%
Epoch: 15  trainAccuracy: 64.910%
Epoch: 16  trainAccuracy: 64.720%
Epoch: 17  trainAccuracy: 65.560%
Epoch: 18  trainAccuracy: 65.130%
Epoch: 19  trainAccuracy: 65.200%
Epoch: 20  trainAccuracy: 65.430%
Epoch: 21  trainAccuracy: 66.180%
Epoch: 22  trainAccuracy: 65.530%
Epoch: 23  trainAccuracy: 66.520%
Epoch: 24  trainAccuracy: 65.900%
Epoch: 25  trainAccuracy: 66.680%
Epoch: 26  trainAccuracy: 66.510%
Epoch: 27  trainAccuracy: 66.600%
Epoch: 28  trainAccuracy: 66.160%
Epoch: 29  trainAccuracy: 